### Consolidate Yloader OHLCV Data

This notebook reads all individual ticker `.csv` files generated by the `Yloader` application, combines them into a single, analysis-ready DataFrame, and saves it in the efficient Parquet format.

**Workflow:**

1.  **Prerequisite:** Run `Yloader` to download OHLCV data. This should create multiple `.csv` files (e.g., `AAPL.csv`, `GOOG.csv`) in a dedicated directory.
2.  **Process & Combine:** The notebook scans the source directory, reads each CSV, and consolidates them into a single pandas DataFrame with a `(Ticker, Date)` MultiIndex.
3.  **Save:** The final DataFrame is saved as a single `.parquet` file for fast loading in subsequent analysis notebooks.
4.  **Verify:** The saved Parquet file is read back to confirm its integrity.

### Setup and Configuration

**This is the only cell you need to modify.** Adjust the paths and column definitions below to match your project structure.

In [1]:
import sys
import pandas as pd
from pathlib import Path
from typing import List, Optional

# --- Configuration ---

# 1. Set the directory where your Yloader CSV files are located.
#    This uses Path.home() to be portable across different computers and OS.
YLOADER_DATA_DIR = Path.home() / "Desktop" / "yloader"

# 2. Define the destination path for the final combined Parquet file.
#    It's good practice to save data outputs within the project's data folder.
#    Assuming a project structure where this notebook is in a `notebooks` subdir.
# ROOT_DIR = Path.cwd().parent if Path.cwd().name == 'notebooks' else Path.cwd()
# DESTINATION_PARQUET_PATH = ROOT_DIR / "data" / "df_OHLCV_stocks_etfs.parquet"
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent
SRC_DIR = ROOT_DIR / "src"
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))
DESTINATION_PARQUET_PATH_OHLCV = ROOT_DIR / "data" / "df_OHLCV_stocks_etfs.parquet"
DESTINATION_PARQUET_PATH_INDICES = ROOT_DIR / "data" / "df_indices.parquet"

# 3. Define the column names for the CSV files.
#    Assumes CSVs have no header and columns are in this fixed order.
#    The first column is always assumed to be 'Date'.
CANONICAL_COLUMN_NAMES = ["Adj Open", "Adj High", "Adj Low", "Adj Close", "Volume"]

# --- Verification ---
print(f"Project Root Directory set to: {ROOT_DIR}")
print(f"Reading Yloader CSVs from: {YLOADER_DATA_DIR}")
print(f"Ticker OHLCV output will be saved to: {DESTINATION_PARQUET_PATH_OHLCV}")
print(f"Indices Output will be saved to: {DESTINATION_PARQUET_PATH_INDICES}")

# Set pandas display options for better readability
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 1000)

Project Root Directory set to: c:\Users\ping\Files_win10\python\py311\stocks
Reading Yloader CSVs from: C:\Users\ping\Desktop\yloader
Ticker OHLCV output will be saved to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet
Indices Output will be saved to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_indices.parquet


### Step 1: Process and Combine CSV Data

This cell defines and executes the core logic to read all individual CSV files and merge them into a single, multi-indexed DataFrame.

In [2]:
from pathlib import Path
from datetime import datetime
from typing import List, Optional

import pandas as pd


def process_yloader_csvs(
    data_dir: Path,
    canonical_cols: List[str],
    anchor_ticker: str = "SPY",
) -> Optional[pd.DataFrame]:
    """
    Reads and combines CSV files from the Yloader output directory whose
    modification *date* (ignoring time-of-day) matches that of the specified
    anchor ticker file.

    Args:
        data_dir (Path):
            Directory containing the Yloader CSV files.
        canonical_cols (List[str]):
            Expected data column names, **excluding** 'Date'.
        anchor_ticker (str, optional):
            Ticker whose modification date is used as the reference.
            Defaults to "SPY".

    Returns:
        Optional[pd.DataFrame]:
            A sorted, multi-indexed DataFrame (Ticker, Date) or None if
            no data could be processed.
    """
    if not data_dir.is_dir():
        print(f"Error: Directory not found: {data_dir}")
        return None

    # ------------------------------------------------------------------ #
    # 1. Determine the reference date (modification date of anchor file) #
    # ------------------------------------------------------------------ #
    anchor_file = data_dir / f"{anchor_ticker.upper()}.csv"
    if not anchor_file.exists():
        print(f"Anchor file not found: {anchor_file}")
        return None

    anchor_date = datetime.fromtimestamp(anchor_file.stat().st_mtime).date()
    print(f"\nProcess csv files with this date: {anchor_date}")

    # ---------------------------------------------------------- #
    # 2. Collect CSV files whose modification date matches anchor #
    # ---------------------------------------------------------- #
    csv_files = [
        f
        for f in data_dir.glob("*.csv")
        if datetime.fromtimestamp(f.stat().st_mtime).date() == anchor_date
    ]

    if not csv_files:
        print(f"No CSV files found with the same modification date as {anchor_ticker}.")
        return None

    print(
        f"Found {len(csv_files)} CSV files with modification date "
        f"{anchor_date} (anchor: {anchor_ticker})..."
    )

    # ---------------------------------------------------------- #
    # 3. Load and combine the matching files                     #
    # ---------------------------------------------------------- #
    all_dataframes = []
    tickers_list = []
    expected_csv_cols = ["Date"] + canonical_cols

    for file_path in csv_files:
        ticker = file_path.stem
        try:
            df_temp = pd.read_csv(
                file_path,
                header=None,
                names=expected_csv_cols,
                parse_dates=["Date"],
                index_col="Date",
            )

            if df_temp.empty:
                print(f"Warning: File {file_path.name} is empty. Skipping.")
                continue

            all_dataframes.append(df_temp)
            tickers_list.append(ticker)

        except Exception as e:
            print(f"Error processing {file_path.name}: {e}. Skipping.")

    if not all_dataframes:
        print("No data was successfully loaded. Aborting.")
        return None

    multi_index_df = pd.concat(
        all_dataframes, keys=tickers_list, names=["Ticker", "Date"]
    )
    multi_index_df.sort_index(
        level=["Ticker", "Date"], ascending=[True, False], inplace=True
    )

    print("\nCSV processing complete. DataFrame created successfully.")
    return multi_index_df

In [3]:
# # --- Execute Step 1 ---
# final_df = process_yloader_csvs(YLOADER_DATA_DIR, CANONICAL_COLUMN_NAMES)

# if final_df is not None:
#     # chronological sort final_df
#     if not final_df.index.is_monotonic_increasing:
#         print(f'\nsorting final_df chronologically ...')
#         final_df.sort_index(inplace=True)
#     else:
#         print(f'\nfinal_df is sorted chronologically')

#     print("\n--- DataFrame Info ---")
#     final_df.info()

#     print("\n--- First 5 Rows of Combined DataFrame ---")
#     display(final_df.tail())

### Step 2: Save DataFrame to Parquet File

This step saves the combined DataFrame to a Parquet file, which is highly efficient for storage and subsequent loading.


In [4]:
import numpy as np
import pandas as pd
import gc
from pathlib import Path
from typing import Optional, Tuple

# ==========================================
# Helper Functions
# ==========================================


def separate_indices_from_tickers(
    df: pd.DataFrame,
) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Efficiently splits a MultiIndex DataFrame into two:
    1. Regular tickers
    2. Indices (tickers starting with '^')

    Uses underlying MultiIndex codes for O(N_unique) performance instead of O(N_rows).
    """
    if df is None or df.empty:
        return pd.DataFrame(), pd.DataFrame()

    print("\nSeparating indices from regular tickers...")

    # 1. Access underlying structure
    # levels: unique strings, codes: integer map to those strings
    levels = df.index.levels[0]
    codes = df.index.codes[0]

    # 2. Identify unique index tickers (vectorized string op on small array)
    is_index_ticker = levels.str.startswith("^")

    # 3. Create boolean mask using integer codes (fast integer comparison)
    target_codes = np.where(is_index_ticker)[0]
    mask = np.isin(codes, target_codes)

    # 4. Split and Copy
    # We use copy() so the new DFs own their data, allowing garbage collection of the original
    df_indices = df.loc[mask].copy()
    df_OHLCV = df.loc[~mask].copy()

    # 5. Clean up metadata
    # Remove unused levels so df_OHLCV doesn't carry metadata about '^VIX', etc.
    df_indices.index = df_indices.index.remove_unused_levels()
    df_OHLCV.index = df_OHLCV.index.remove_unused_levels()

    print(
        f"Separation complete: {len(df_OHLCV):,} regular rows, {len(df_indices):,} index rows."
    )
    return df_OHLCV, df_indices


def save_dataframe_to_parquet(df: Optional[pd.DataFrame], dest_path: Path):
    """
    Saves a DataFrame to a Parquet file using pyarrow/zstd.
    """
    if df is None or df.empty:
        print(f"Skipping save: DataFrame for {dest_path.name} is empty or None.")
        return

    try:
        dest_path.parent.mkdir(parents=True, exist_ok=True)
        df.to_parquet(dest_path, engine="pyarrow", compression="zstd", index=True)
        print(f"Successfully saved to: {dest_path}")
    except Exception as e:
        print(f"Error saving {dest_path}: {e}")


# ==========================================
# Main Execution
# ==========================================

# 1. Load Data
print(f"==== Takes about 2.5 minutes to run ====")
final_df = process_yloader_csvs(YLOADER_DATA_DIR, CANONICAL_COLUMN_NAMES)

if final_df is not None and not final_df.empty:

    # 2. Sort Chronologically (Optimization: Sort once before splitting)
    if not final_df.index.is_monotonic_increasing:
        print(f"\nSorting combined DataFrame chronologically...")
        final_df.sort_index(inplace=True)
    else:
        print(f"\nCombined DataFrame is already sorted.")

    print("\n--- Combined DataFrame Info ---")
    final_df.info()

    # 3. Split DataFrames
    df_OHLCV, df_indices = separate_indices_from_tickers(final_df)

    # 4. Memory Cleanup
    # Critical for large datasets: delete the massive original DF and force GC
    print("\nReleasing memory from combined DataFrame...")
    del final_df
    gc.collect()

    # 5. Save Outputs
    save_dataframe_to_parquet(df_OHLCV, DESTINATION_PARQUET_PATH_OHLCV)
    save_dataframe_to_parquet(df_indices, DESTINATION_PARQUET_PATH_INDICES)

    # Display preview (using display if in Jupyter, else print)
    try:
        print("\n--- Preview: Regular Tickers (df_OHLCV) ---")
        display(df_OHLCV.tail(3))
        print("\n--- Preview: Indices (df_indices) ---")
        display(df_indices.tail(3))
    except NameError:
        print(df_OHLCV.tail(3))
        print(df_indices.tail(3))

else:
    print("No data loaded from process_yloader_csvs.")

==== Takes about 2.5 minutes to run ====

Process csv files with this date: 2026-02-07
Found 1594 CSV files with modification date 2026-02-07 (anchor: SPY)...

CSV processing complete. DataFrame created successfully.

Sorting combined DataFrame chronologically...

--- Combined DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
MultiIndex: 9606332 entries, ('A', Timestamp('1999-11-18 00:00:00')) to ('^VIX', Timestamp('2026-02-06 00:00:00'))
Data columns (total 5 columns):
 #   Column     Dtype  
---  ------     -----  
 0   Adj Open   float64
 1   Adj High   float64
 2   Adj Low    float64
 3   Adj Close  float64
 4   Volume     int64  
dtypes: float64(4), int64(1)
memory usage: 403.8+ MB

Separating indices from regular tickers...
Separation complete: 9,467,170 regular rows, 139,162 index rows.

Releasing memory from combined DataFrame...
Successfully saved to: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet
Successfully saved to: c:\Users\ping\Fil

Adj Open  Adj High  Adj Low  Adj Close   Volume
Ticker Date                                                       
ZWS    2026-02-04     48.59     52.77    47.30      51.77  2926100
       2026-02-05     51.25     52.02    50.54      51.11  1772300
       2026-02-06     51.50     52.84    51.43      52.78  1843700


--- Preview: Indices (df_indices) ---


Adj Open  Adj High  Adj Low  Adj Close  Volume
Ticker Date                                                      
^VIX   2026-02-04     17.63     21.24    17.47      18.64       0
       2026-02-05     19.24     23.10    19.06      21.77       0
       2026-02-06     21.24     21.49    19.99      20.37       0

### Step 3: Verify the Saved File (Optional)

This final step reads the Parquet file back into a new DataFrame to ensure the data was saved correctly.

In [5]:
print(f"--- Verifying the saved file at: {DESTINATION_PARQUET_PATH_OHLCV} ---")

try:
    if DESTINATION_PARQUET_PATH_OHLCV.exists():
        verified_df = pd.read_parquet(DESTINATION_PARQUET_PATH_OHLCV)
        print("Verification successful! File read back into memory.")

        print("\n--- First 5 Rows of Verified DataFrame ---")
        display(verified_df.head())

        # Optional: Check a specific ticker
        if not verified_df.empty:
            example_ticker = verified_df.index.get_level_values("Ticker")[0]
            print(f"\n--- Data for first available ticker '{example_ticker}' ---")
            display(verified_df.loc[example_ticker].head())

    else:
        print("Error: The output file was not found at the specified path.")

except Exception as e:
    print(f"An error occurred during verification: {e}")

--- Verifying the saved file at: c:\Users\ping\Files_win10\python\py311\stocks\data\df_OHLCV_stocks_etfs.parquet ---
Verification successful! File read back into memory.

--- First 5 Rows of Verified DataFrame ---


Adj Open  Adj High  Adj Low  Adj Close    Volume
Ticker Date                                                        
A      1999-11-18   27.1966   29.8864  23.9091    26.3000  74849975
       1999-11-19   25.6649   25.7023  23.7970    24.1333  18230878
       1999-11-22   24.6936   26.3000  23.9465    26.3000   7871813
       1999-11-23   25.4034   26.0759  23.9091    23.9091   7151079
       1999-11-24   23.9838   25.0672  23.9091    24.5442   5795947


--- Data for first available ticker 'A' ---


,Adj Open,Adj High,Adj Low,Adj Close,Volume
Date,,,,,
1999-11-18,27.1966,29.8864,23.9091,26.3000,74849975
1999-11-19,25.6649,25.7023,23.7970,24.1333,18230878
1999-11-22,24.6936,26.3000,23.9465,26.3000,7871813
1999-11-23,25.4034,26.0759,23.9091,23.9091,7151079
1999-11-24,23.9838,25.0672,23.9091,24.5442,5795947


In [6]:
verified_df.loc["AAPL"]

,Adj Open,Adj High,Adj Low,Adj Close,Volume
Date,,,,,
1980-12-12,0.098390,0.098817,0.098390,0.098390,611849047
1980-12-15,0.093684,0.093684,0.093257,0.093257,229439721
1980-12-16,0.086839,0.086839,0.086412,0.086412,137921043
1980-12-17,0.088550,0.088978,0.088550,0.088550,112762105
1980-12-18,0.091118,0.091545,0.091118,0.091118,95814196
...,...,...,...,...,...
2026-02-02,260.030000,270.490000,259.210000,270.010000,73913400
2026-02-03,269.200000,271.880000,267.610000,269.480000,64394700
2026-02-04,272.290000,278.950000,272.290000,276.490000,90545700


#################  
#################  